<a href="https://colab.research.google.com/github/Tanmay-Somani/Wh020_Linktree/blob/main/Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#book recommender system using collaborative clustering


In [169]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [170]:
books=pd.read_csv("sample_data/BX-Books.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-170-b3d0ab71e29e>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books=pd.read_csv("sample_data/BX-Books.csv",sep=';',error_bad_lines=False ,encoding='latin-1')
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw

In [171]:
books=books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [172]:
books.rename(columns={
    "Book-Title":"title",
    "Books-Author":"author",
    "Year-Of-Publication":"year",
    "Image-URL-L":"img_url"
},inplace=True)

In [173]:
users=pd.read_csv("sample_data/BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-173-9ca8b7c8fe35>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users=pd.read_csv("sample_data/BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In [174]:
ratings=pd.read_csv("sample_data/BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

<ipython-input-174-252b9ded5838>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings=pd.read_csv("sample_data/BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In collaborative system it is important for a user to give you ratings.

In [175]:
ratings.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

In [176]:
x=ratings["user_id"].value_counts()>200

In [177]:
y=x[x].index

In [178]:
ratings=ratings[ratings['user_id'].isin(y)]

In [179]:
ratings_with_books=ratings.merge(books,on="ISBN")

In [180]:
num_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [181]:
num_rating.rename(columns={"rating":"num_of_rating"},inplace=True)

In [182]:
final_rating=ratings_with_books.merge(num_rating,on ='title')

In [183]:
final_rating=final_rating[final_rating["num_of_rating"]>=50]

In [184]:
final_rating.sample(10)

,user_id,ISBN,rating,title,Book-Author,year,Publisher,num_of_rating
10354,46398,0553280368,0,C Is for Corpse (Kinsey Millhone Mysteries (Pa...,Sue Grafton,1987,Bantam,93
80170,239584,1565117794,0,Life of Pi,Yann Martel,2003,Highbridge Audio,185
18641,166123,0425147363,0,Tom Clancy's Op-Center (Tom Clancy's Op Center...,Tom Clancy,1995,Berkley Publishing Group,63
82104,8245,1569471169,7,Falling Angels,Barbara Gowdy,1998,Soho Press,61
150812,68555,0060959037,0,Prodigal Summer: A Novel,Barbara Kingsolver,2001,Perennial,50
2046,108352,0316602051,0,The Jester,James Patterson,2003,"Little, Brown",68
8650,143415,0446611212,0,Violets Are Blue,James Patterson,2002,Warner Vision,150
20392,198711,0451162072,0,Pet Sematary,Stephen King,1994,Signet Book,119
42732,87555,0553272535,0,Night,Elie Wiesel,1982,Bantam Books,76
25197,129358,0345313860,0,"The Vampire Lestat (Vampire Chronicles, Book II)",ANNE RICE,1986,Ballantine Books,123


In [185]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [186]:
book_pivot=final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [187]:
book_pivot.fillna(0,inplace=True)

In [188]:
from scipy.sparse import csr_matrix

In [189]:
book_sparse=csr_matrix(book_pivot)

In [190]:
from sklearn.neighbors import NearestNeighbors

In [191]:
model=NearestNeighbors(algorithm='brute')


In [192]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [193]:
distance,suggestion=model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)


In [194]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [195]:
suggestion

array([[237, 240, 238, 241, 184, 536]])

In [196]:
for i in range(len(suggestion)):
  print(book_pivot.index[suggestion[i]])
  #to see the recommendation

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [197]:
book_pivot.index[3]

'4 Blondes'

In [198]:
books_name=book_pivot.index

In [199]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(books_name,open('artifacts/mbook_name.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))


In [215]:
def recommend_me_the_book(book_name):
  try:
    book_id=np.where(book_pivot.index ==book_name)[0][0]
    distance,suggestion=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
  
    for i in range(len(suggestion)):
      books=book_pivot.index[suggestion[i]]
      for j in books:
        print(j)
  except IndexError:
    print("The book doesnt exists in the database")
  


In [217]:
book_name=''
recommend_me_the_book(book_name)

The book doesnt exists in the database
